## Relative Abundance Stacked Barplot

In [154]:
import pandas as pd
import biom
from biom import load_table
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np

In [155]:
# Set taxon level
# taxon_level = 'genus'
taxon_level = 'species'

In [156]:
# Load biom table and metadata
biom_path = f"../tables/3_228413_gOTU_micov75%_{taxon_level}_collapsed.biom"
metadata = pd.read_csv("../metadata/marta_metadata.tsv", sep='\t')

In [157]:
# View biom table as convert to tsv
biom_table = load_table(biom_path)
df = pd.DataFrame(biom_table.to_dataframe()).T
df.index = df.index.astype(str)
df.index = df.index.str.replace('16207.', '')
df = df.loc[metadata['sample_name']]
# df = df.rename(columns={"s__": "s__Unknown"})
if 's__' in df.columns:
    df = df.drop('s__', axis=1)
    
# Save df as tsv
df.to_csv(f"../tables/3_228413_gOTU_micov75%_{taxon_level}_collapsed.tsv", sep="\t")

In [158]:
# Input/output paths
infile = f"../tables/3_228413_gOTU_micov75%_{taxon_level}_collapsed.tsv"
outfile = f"../tables/4_228413_gOTU_micov75%_{taxon_level}_collapsed_relab.tsv"

# Load table: rows = samples, columns = taxa
df = pd.read_csv(infile, sep="\t", index_col=0)

# Convert to relative abundance (per sample)
df_rel = df.div(df.sum(axis=1), axis=0)

# Load metadata and index by sample_name
md2 = metadata.set_index("sample_name")

# Update index to:  (sample_number) treatment [microbiome_type]
sample_name_to_index = {
    sn: f"({md2.loc[sn, 'sample_number']}) {md2.loc[sn, 'treatment']} [{md2.loc[sn, 'microbiome_type']}]"
    for sn in md2.index
}

df_rel.index = df_rel.index.map(lambda x: sample_name_to_index[x])

# Save as TSV
df_rel.to_csv(outfile, sep="\t")

In [159]:
# Get all unique species across the whole dataframe
all_species_global = df_rel.columns.tolist()

# Create a consistent color palette for all species
colors = cm.get_cmap('tab20')(np.linspace(0, 1, len(all_species_global)))
species_colors_global = dict(zip(all_species_global, colors))

# Loop through each group
for group in md2['plot_group'].unique():
    # Get samples in this group
    samples_in_group = md2[md2['plot_group'] == group].index
    # Map to new index names
    new_index_names = [sample_name_to_index[s] for s in samples_in_group if s in sample_name_to_index]
    
    # Subset df_rel
    df_group = df_rel.loc[df_rel.index.isin(new_index_names)]
    
    # Plot with independent sorting for each bar
    plt.figure(figsize=(20, 10))
    
    # Track which species have been added to legend
    legend_added = set()
    
    # For each sample, sort species by abundance and plot
    for i, sample in enumerate(df_group.index):
        # Sort species for this sample in descending order (most abundant first)
        sample_sorted = df_group.loc[sample].sort_values(ascending=False)
        
        # Initialize bottom position for stacking
        bottom = 0
        
        # Plot each species as a bar segment
        for species, abundance in sample_sorted.items():
            if abundance > 0:
                # Add to legend only once
                label = species if species not in legend_added else ""
                if label:
                    legend_added.add(species)
                
                plt.bar(i, abundance, bottom=bottom, width=0.8,
                        color=species_colors_global[species], label=label)
                bottom += abundance
    
    # Set x-axis labels and positions
    plt.xticks(range(len(df_group.index)), df_group.index, rotation=90, ha="center", fontsize=16)
    plt.ylabel("Relative Abundance", fontsize=16)
    plt.title(f"Microbiome Composition ({taxon_level}): {group}", fontsize=22)
    plt.legend(bbox_to_anchor=(1, 1), loc="best", title=taxon_level, title_fontsize=16, fontsize=14)
    plt.tight_layout()

    if group == 'Em23 + MC mouse stool':
        plt.text(
            1.2,
            -0.8,
            "Data by Yang Chen",
            fontsize=10,
            color='grey',
            ha='right',
            va='bottom',
            transform=plt.gca().transAxes
        )
    elif group == 'Em19 mouse skin':
        plt.text(
            1.2,
            -0.4,
            "Data by Yang Chen",
            fontsize=10,
            color='grey',
            ha='right',
            va='bottom',
            transform=plt.gca().transAxes
        )
    elif group == 'Em19 mouse stool':
        plt.text(
            1.2,
            -0.4,
            "Data by Yang Chen",
            fontsize=10,
            color='grey',
            ha='right',
            va='bottom',
            transform=plt.gca().transAxes
        )            
    else:
        plt.text(
            1.2,
            -0.5,
            "Data by Yang Chen",
            fontsize=10,
            color='grey',
            ha='right',
            va='bottom',
            transform=plt.gca().transAxes
        )

    
    # Save figure
    plt.savefig(f"../figures/relative_abundance_sorted_{group}_{taxon_level}.png", dpi=600, bbox_inches='tight')
    plt.close()

/var/folders/22/yck9vwx53w1c38tvj_c0_tz00000gn/T/ipykernel_60616/3861053802.py:5: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  colors = cm.get_cmap('tab20')(np.linspace(0, 1, len(all_species_global)))
